In [1]:
from keras.models import Sequential

Using TensorFlow backend.


TODO import CSV data and transform
TODO: get more data from the card page (http://www.wri.org/sites/default/files/Aqueduct_Global_Maps_2.1.pdf)
    - Reason: WDIData only has indicators for each year. This is too little to do actual machine-learning